In [ ]:
# install the library
from zoraspeech.listener import Listener
from zoraspeech.architectures.cnn.cnn import ConvModel
from zoraspeech.interpreters.cnn.layer_visualizations import LightweightVisualizer

import numpy as np
import soundfile as sf

import torch as t
import torchaudio
import matplotlib.pyplot as plt
from IPython.display import display, Audio
import random
import glob
#import einops

In [ ]:
# set up autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# create a listener
listener = Listener(
    model_architecture=ConvModel(), 
    model_weights='/Users/jo/Documents/zora/src/zoraspeech/weights/audrey_model_weights_2024-10-26.pth',
    interpreter=LightweightVisualizer()
    )
listener.load()

In [ ]:
# load in the random file
random_file = '/Users/jo/Documents/listening_and_speaking_machines/listening_machines/speech_digit_dataset/test_6.wav'

recorded_audio, sample_rate = torchaudio.load(random_file)
print(sample_rate)

# visualize
print(recorded_audio.shape)
print(len(recorded_audio[0]))
# pad the audio to the longest audio file length

#recorded_audio = np.array(recorded_audio)[0]
current_size = len(recorded_audio[0])

longest_audio_file_length = 17916

if current_size > longest_audio_file_length:
    print("Audio is too long, truncating")
    padded_audio = recorded_audio[:, :longest_audio_file_length]
    current_size = longest_audio_file_length
else:
    print("Audio is short enough, padding")

    pad_size = longest_audio_file_length - current_size
    print("Pad size: ", pad_size)
    left_pad = pad_size // 2
    print("Left pad: ", left_pad)
    right_pad = pad_size - left_pad
    print("Right pad: ", right_pad)
    padded_audio = np.pad(recorded_audio[0], (left_pad, right_pad), mode='constant')
    print("Padded audio shape: ", padded_audio.shape)
    print(padded_audio.shape)

display(Audio(padded_audio, rate=sample_rate))

padded_audio_array = np.array(padded_audio)
audio = t.tensor([padded_audio_array])

# display spectrogram
spec = torchaudio.transforms.MelSpectrogram()(audio)


In [ ]:
# have listener object "listen" to audio spectrogram
listener.listen(spec)

In [ ]:
# visualize the activation
listener.interpreter.interpret(spec)

In [ ]:
"""
listener.interpret.list()

listener.interpret("cnn_visualization")

--> generate activation maps from cnn model


listener.interpreter.list()
listener.interpreter.interpret("cnn_visualization")

--> generate activation maps from cnn model + sonification

listener.interpreter.interpret("mech_interp")
"""